In [8]:
# This function gives the digits of a number n in a given base padded to padto.
# These digits build the "data" for the pitches. Because we are modulo 8 (or 12 if you prefer),
# a function f which is defined over these digits-data will eventually repeat itself modulo 8 and parts of it
# will sound similar to the listener of the melody.
def digits(n,base,padto=None):
    q = n
    ret = []
    while q != 0:
        q, r = q//base,q%base # Divide by 10, see the remainder
        ret.insert(0, r) # The remainder is the first to the right digit
    if padto is None:
        return ret
    for i in range(padto-len(ret)):
        ret.insert(0,0)
    #ret.extend((padto-len(ret))*[0])    
    return ret

print(digits(8,2,10))

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [159]:
# These functions define some functions wich are invariant under some finite group and will get applied to the digits data.
# If you want a new melody, simply define a new symmetric (under some group) function here and apply it to your digits data.
# The first four are similar if you pay attention to the definition. 
# ( I would suggest to use sagemath for this purpose https://sagecell.sagemath.org/ ) or do the computations by hand using the Reynolds operator,
# as this is done only once.
# The abc function is called so because it comes up at the abc conjecture ab(a+b)/gcd(a,b)^3 .
import math
funcTirana = lambda a: 2*int(math.pow(a[2],2)) + 2*int(math.pow(a[3],2)) + 2*int(math.pow(a[4],2)) + 3*a[0] + 3*a[1]
funcTirana2 = lambda a: -(2*a[2]**2 + 2*a[3]**2 + 2*a[4]**2) + 3*a[0] + 3*a[1]
funcTirana3 = lambda a: (2*a[2]**2 + 2*a[3]**2 + 2*a[4]**2) + -(3*a[0] + 3*a[1])
funcTirana4 = lambda a: -(2*a[2]**2 + 2*a[3]**2 + 2*a[4]**2) + -(3*a[0] + 3*a[1])
funcABC = lambda a: int(a[0]*a[1]*(a[0]+a[1])/math.gcd(a[0],a[1])**3)
funcs = [funcTirana,funcTirana2,funcTirana3,funcTirana4]
func = lambda a: a[0]**2*a[1]+a[1]**2*a[0] #int((a[0]**5*a[1]**1+a[0]**4*a[1]**2+a[0]**2*a[1]**4+a[0]**1*a[1]**5)/math.gcd(a[0],a[1])**6)


In [51]:
import math

# These functions which come below are used to generate the bars / rhythm. The idea is to map natural numbers in
# different ways to binary trees (/ bars(4:4)) to get different rhythms.

# This function computes the divisors of a number. The algorithm is brute force as I use it only for small numbers.
# 
def divisors(n):
    dd = []
    for k in range(1,n+1):
        if n%k == 0:
            dd.append(k)
    return dd            

# This function constructs a binary tree given a natural number using the divisors of a number n = e*d (where e <= sqrt(n) and d>= sqrt(n))
def divisorTree(n):
    if n==1 or len(divisors(n))==2:
        return []
    else:
        dd = divisors(n)
        l = max([ x for x in dd if x <= math.sqrt(n*1.0)])
        r = min([ x for x in dd if x >= math.sqrt(n*1.0)])
        return [divisorTree(l),divisorTree(r)]

# Similar to the function above this function generates a binary tree from a natural number, decomposing a natural number as a sum:
# n = (n//2) + (n-(n//2))
def sumTree(n)    :
    if n==1:
        return []
    else:
        return [sumTree(int(n//2)),sumTree(n-int(n//2))]

In [2]:
# This function assigns to each binary tree a duration based on the height of the leafs in the tree:
# height 0 = 1
# height 1 = 2
# height 2 = 4
# height 3 = 8
# height n = 2**n etc. (The tree might be one of divisorTree or sumTree)
def getDurationsFromTree(tree):
    # Identify leaves by their length
    if len(tree) == 0:
        return [1]
    # Take all branches, get the paths for the branch, and prepend current
    dd = []
    for t in tree:
        dd.extend([2*d for d in getDurationsFromTree(t)])
    return dd

In [ ]:
import mingus.core.notes as notes
import mingus.core.scales as scales
cmajor = scales.Major("C").ascending()
nTracks = 4 # we have four tracks (piano bass, piano g-clef, violoncello, vibraphone)
mybars = []
for k in range(nTracks):
    mybars.append([])

# Those permutations define in the piece ( https://musescore.com/user/37663311/scores/6644743 )    
# which two instruments will have the "tempo leads" corresponding to 3 and 4 and wich instruments play slow corresponding to 
# 1 and 2.
# By permuting the instruments having the tempo lead a variation arises:
permutations = [(1, 2, 3, 4), (3, 4, 1, 2), (2, 4, 1, 3), (4, 1, 2, 3), (3, 1, 4, 2),  (1, 3, 2, 4), (1, 2, 3, 4)] 

# These tree functions are important as they assign to each natural number a tree.
# 1 is for the first instrument (after permutation)
# 2 is for the second instrument (again after permutation)
# 3 and 4 similar:
treeFuncs = [lambda n : divisorTree(n), lambda n: divisorTree(2*n), lambda n : sumTree(n%16+1), lambda n: sumTree(n%16+8)]

# I use only cmajor at the moment
mynotes = cmajor #[notes.int_to_note(i) for i in range(8)]
for tt in range(nTracks):
    c = 0 # This counter is important for each instrument as it gives the "digits-data" which we apply the symmetric function to, 
    #to get the pitches:
    for n in range(1,6*16+1): # we have 6+1 permutations. Each interval is 16 bars long where the tempo grows slowly
        print(symgr4[n//16])
        durations = getDurationsFromTree(treeFuncs[permutations[n//16][tt]-1](n)) # this is a bit difficult to explain...
        pitches = []
        volumes = []
        for d in durations:
            c+= 1
            pitch = cmajor[((-1)**n*funcTirana2([d+1 for d in digits(c,8,5)]))%8] # here we apply digits to the counter c to get the digits data, and the apply funcTirana2*(-1)**n mod 8 to get the pitches in cmajor
            pitches.append(pitch)
            volumes.append(int(min(127,max(100,64))))    

        bar = list(zip(pitches,durations,volumes)) # one bar is created and saved in mybars and printed
        #bar = (getNotesFromDivisorTree(divisorTree(n),mynotes))
        print(bar)
        mybars[tt].append(bar)
    
#zipped = sorted(zipped)    
print(mybars)    

In [104]:
# In the following part, 
# I put the bars together to tracks (each instrument has one track) and write the midi, 
# I also define the octaves manually for each instrument:


In [161]:
from mingus.containers import Note
from mingus.containers import NoteContainer
from mingus.containers import Bar
from mingus.containers import Track
from mingus.containers import Composition
from mingus.containers.instrument import MidiInstrument
from mingus.midi import midi_file_out
from mingus.containers.instrument import Instrument, Piano, Guitar
import mingus.core.notes as notes
import mingus.core.scales as scales
import mingus.extra.lilypond as LilyPond

cmajor = scales.Major("C").ascending()
#cmajor.extend(scales.Major("D").ascending())

print(cmajor)

nc = NoteContainer()

p = Piano()
s = MidiInstrument("Alto Sax")
s.instrument_nr = 65
g = MidiInstrument("Acoustic Guitar")
g.instrument_nr = 25
o = MidiInstrument("Rock Organ")
d = MidiInstrument("Steel Drums")
m = MidiInstrument("Musical Box")
m.instrument_nr = 11
d.instrument_nr = 115
v = MidiInstrument("Viola")
v.instrument_nr = 42

track1=Track(b)
track2=Track(o)
track3 = Track(p)
#tracks = [track1,track2,track3]
tracks = [Track(p),Track(p),Track(v),Track(m)] #,Track(p)]



bar=Bar(meter=(4,4))
counter = 0
#ata = [(min(max(x+i,0),6),min(max(y+j,0),7)) for i in range(-1,2) for j in range(-1,2) for x in range(7) for y in range(8)]
sumDurations = 0
noteCont = []
N = 32
mods = [3,8]
octaves = [2,4,3,4]
for tt in range(len(tracks)):
    track = tracks[tt]
    counter = 0
    for t in range(len(mybars[tt])):#len(zips[tt])):
        bar = Bar(meter=(4,4))
        mybar = mybars[tt][t]
        for i in range(len(mybar)):
            pitch,duration,volume = mybar[i]
            octave = octaves[tt]
            note = Note(name=pitch,octave=octave,velocity=volume,channel=10)
            placed = bar.place_notes(note,duration=duration)        
            print(note,duration,placed)
        track.add_bar(bar)
        
            
compo=Composition()
for track in tracks:
    print(track)
    compo.add_track(track)
    
    
name = "composition"
bpm = 80
midi_file_out.write_Composition(name+".mid",compo,bpm)



  

['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C']
'B-2' 1 True
'C-2' 1 True
'B-2' 1 True
'C-2' 2 True
'E-2' 2 True
'C-2' 1 True
'E-2' 2 True
'E-2' 2 True
'G-2' 1 True
'E-2' 2 True
'G-2' 4 True
'E-2' 4 True
'G-2' 2 True
'B-2' 2 True
'G-2' 2 True
'C-2' 2 True
'B-2' 1 True
'C-2' 2 True
'E-2' 4 True
'C-2' 4 True
'B-2' 1 True
'C-2' 2 True
'E-2' 2 True
'B-2' 2 True
'G-2' 2 True
'E-2' 1 True
'G-2' 2 True
'B-2' 2 True
'G-2' 2 True
'E-2' 4 True
'G-2' 4 True
'C-2' 4 True
'B-2' 4 True
'C-2' 4 True
'B-2' 4 True
'C-2' 4 True
'E-2' 4 True
'C-2' 4 True
'E-2' 8 True
'E-2' 8 True
'G-2' 4 True
'B-2' 8 True
'G-2' 8 True
'B-2' 4 True
'G-2' 8 True
'B-2' 8 True
'G-2' 4 True
'C-2' 8 True
'E-2' 8 True
'C-2' 8 True
'E-2' 8 True
'C-2' 8 True
'E-2' 8 True
'C-2' 8 True
'B-2' 8 True
'B-2' 8 True
'G-2' 8 True
'B-2' 8 True
'G-2' 8 True
'B-2' 8 True
'G-2' 8 True
'E-2' 8 True
'G-2' 8 True
'E-2' 8 True
'G-2' 8 True
'E-2' 8 True
'G-2' 8 True
'E-2' 8 True
'G-2' 16 True
'E-2' 16 True
'G-2' 8 True
'G-2' 8 True
'E-2' 8 True
'G-2' 16

'F-2' 1 True
'C-2' 4 True
'A-2' 4 True
'C-2' 4 True
'A-2' 4 True
'D-2' 2 True
'D-2' 2 True
'D-2' 4 True
'C-2' 4 True
'D-2' 2 True
'D-2' 2 True
'C-2' 2 True
'C-2' 2 True
'D-2' 2 True
'A-2' 2 True
'F-2' 2 True
'F-2' 4 True
'A-2' 8 True
'F-2' 8 True
'A-2' 4 True
'F-2' 8 True
'A-2' 8 True
'B-4' 1 True
'C-4' 2 True
'E-4' 2 True
'C-4' 2 True
'B-4' 2 True
'C-4' 2 True
'E-4' 4 True
'E-4' 4 True
'G-4' 2 True
'B-4' 2 True
'G-4' 2 True
'E-4' 4 True
'G-4' 4 True
'B-4' 2 True
'G-4' 2 True
'C-4' 4 True
'E-4' 4 True
'C-4' 4 True
'E-4' 4 True
'C-4' 2 True
'B-4' 4 True
'C-4' 4 True
'E-4' 4 True
'E-4' 4 True
'G-4' 2 True
'B-4' 2 True
'G-4' 2 True
'E-4' 4 True
'G-4' 4 True
'E-4' 4 True
'G-4' 4 True
'C-4' 2 True
'B-4' 2 True
'C-4' 4 True
'E-4' 4 True
'C-4' 2 True
'B-4' 2 True
'C-4' 4 True
'B-4' 4 True
'E-4' 8 True
'G-4' 8 True
'E-4' 8 True
'G-4' 8 True
'E-4' 8 True
'G-4' 8 True
'E-4' 8 True
'G-4' 8 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 16 Tru

'F-4' 16 True
'A-4' 16 True
'F-4' 8 True
'A-4' 16 True
'A-4' 16 True
'C-4' 8 True
'A-4' 16 True
'C-4' 16 True
'F-4' 8 True
'D-4' 16 True
'F-4' 16 True
'D-4' 8 True
'A-4' 16 True
'F-4' 16 True
'A-4' 8 True
'F-4' 16 True
'A-4' 16 True
'F-4' 8 True
'A-4' 16 True
'F-4' 16 True
'A-4' 8 True
'C-4' 16 True
'A-4' 16 True
'C-4' 8 True
'A-4' 16 True
'C-4' 16 True
'A-4' 8 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'D-4' 8 True
'F-4' 16 True
'D-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 8 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'C-4' 8 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'A-4' 16 True
'C-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4

'D-3' 4 True
'A-3' 2 True
'A-3' 2 True
'F-3' 8 True
'A-3' 8 True
'F-3' 8 True
'A-3' 8 True
'A-3' 2 True
'F-3' 2 True
'A-3' 4 True
'C-3' 8 True
'A-3' 8 True
'C-3' 4 True
'A-3' 4 True
'D-3' 2 True
'F-3' 4 True
'D-3' 4 True
'F-3' 4 True
'A-3' 8 True
'F-3' 8 True
'A-3' 2 True
'A-3' 4 True
'F-3' 4 True
'A-3' 2 True
'A-3' 4 True
'A-3' 4 True
'C-3' 2 True
'F-3' 4 True
'D-3' 4 True
'F-3' 2 True
'C-3' 1 True
'B-4' 8 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 8 True
'B-4' 16 True
'G-4' 16 True
'E-4' 8 True
'G-4' 8 True
'E-4' 8 True
'G-4' 16 True
'E-4' 16 True
'G-4' 8 True
'C-4' 8 True
'E-4' 8 True
'C-4' 16 True
'E-4' 16 True
'C-4' 8 True
'B-4' 8 True
'C-4' 8 True
'B-4' 16 True
'B-4' 16 True
'G-4' 8 True
'B-4' 16 True
'G-4' 16 True
'B-4' 8 True
'G-4' 16 True
'B-4' 16 True
'G-4' 8 True
'C-4' 16 True
'E-4' 16 True
'C-4' 8 True
'E-4' 16 True
'C-4' 16 True
'E-4' 8 True
'C-4' 16 True
'E-4' 16 True
'E-4' 8 True
'G-4' 16 True
'E-4' 16 True
'G-4' 8 True
'B-4' 16 True
'G-4

'A-4' 8 True
'A-4' 16 True
'C-4' 16 True
'A-4' 8 True
'C-4' 16 True
'A-4' 16 True
'C-4' 8 True
'A-4' 16 True
'C-4' 16 True
'A-4' 8 True
'F-4' 16 True
'A-4' 16 True
'F-4' 8 True
'A-4' 16 True
'F-4' 16 True
'A-4' 8 True
'F-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'C-4' 8 True
'A-4' 16 True
'C-4' 16 True
'F-4' 16 True
'A-4' 16 True
'F-4' 16 True
'A-4' 16 True
'F-4' 8 True
'A-4' 16 True
'F-4' 16 True
'A-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'D-4' 8 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'F-4' 16 True
'D-4' 16 True
'D-4' 16 True
'C-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'A-4' 16 True
'C-4' 16 True
'C-4' 16 True
'D-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'C-4' 16 True
'D-4' 16 True
'

\header { title = "Algorithmic Composition" composer = "Orges Leka" opus = "" } { { b,1 } { c,1 } { b,1 } { c,2 e,2 } { c,1 } { e,2 e,2 } { g,1 } { e,2 g,4 e,4 } { g,2 b,2 } { g,2 c,2 } { b,1 } { c,2 e,4 c,4 } { b,1 } { c,2 e,2 } { b,2 g,2 } { e,1 } { g,2 b,2 } { g,2 e,4 g,4 } { c,4 b,4 c,4 b,4 } { c,4 e,4 c,4 e,8 e,8 } { g,4 b,8 g,8 b,4 g,8 b,8 } { g,4 c,8 e,8 c,8 e,8 c,8 e,8 } { c,8 b,8 b,8 g,8 b,8 g,8 b,8 g,8 } { e,8 g,8 e,8 g,8 e,8 g,8 e,8 g,16 e,16 } { g,8 g,8 e,8 g,16 e,16 g,8 e,8 g,8 e,16 b,16 } { g,8 e,8 g,8 e,16 g,16 e,8 g,16 g,16 b,8 g,16 b,16 } { g,8 e,16 g,16 e,8 b,16 g,16 b,8 g,16 b,16 g,8 b,16 g,16 } { g,8 b,16 g,16 b,8 g,16 b,16 g,8 b,16 e,16 g,16 e,16 g,16 e,16 } { g,8 b,16 g,16 g,16 e,16 g,16 e,16 g,8 e,16 g,16 e,16 c,16 b,16 c,16 } { e,8 c,16 e,16 c,16 e,16 e,16 g,16 e,16 g,16 e,16 g,16 e,16 g,16 c,16 e,16 } { c,16 b,16 c,16 b,16 c,16 b,16 b,16 g,16 b,16 g,16 b,16 g,16 b,16 g,16 c,16 b,16 } { c,4 e,8 c,8 e,4 c,8 e,8 } { b,4 g,4 b,2 } { g,4 e,4 g,2 } { b,2 g,4 c,4 } { 

./convertSong.sh 'composition.mid' 'composition.mp3' '/home/orges/allinkl/website/bilder/017.png_mosaic.jpg' 'composition.mp4'


0